In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=16c509a834f2e254af1ee8468f17d4882f0b90fc31f179313d48c127061fbadc
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .getOrCreate()

In [3]:
# Check spark session
spark

In [4]:
# Mounting google drive so we can access any file inside it
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [42]:
# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

# Create the streaming_df to read from input directory
streaming_df = spark.readStream\
    .format("json") \
    .option("maxFilesPerTrigger", 1) \
    .load("/content/gdrive/MyDrive/input-05111740000038/news/")

In [43]:
# Checking schema of the dataframe
streaming_df.printSchema()

root
 |-- authors: string (nullable = true)
 |-- category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- link: string (nullable = true)
 |-- short_description: string (nullable = true)



In [44]:
print(streaming_df.isStreaming)

True


In [45]:
# Write the output to console sink to check the output

# Start the streaming application to run until the following happens
# 1. Exception in the running program
# 2. Manual Interruption
writing_df = streaming_df.writeStream \
    .format("json") \
    .option("path", "/content/gdrive/MyDrive/output-05111740000038/") \
    .option("checkpointLocation","/content/gdrive/MyDrive/checkpoint") \
    .outputMode("append") \
    .start() \
    .awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [46]:
# Check the data at the output location

out_df = spark.read.json("/content/gdrive/MyDrive/output-05111740000038/")
out_df.show(truncate=False)

AnalysisException: ignored